In [1]:
# Importo le librerie necessarie

import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import numpy as np 

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

f1 = "portieri_19.xlsx"
f2 = "portieri_20.xlsx"
f3 = "portieri_21.xlsx"
f4 = "portieri_22.xlsx"
f5 = "portieri_23.xlsx"

# Elimino le colonne non utili per la nostra analisi
colonne_da_eliminare = ['Player', 'Nation', 'Pos', 'Squad', 'Age', 'MP', 'Starts', 'Min', 'Save%',
                        'CS%', 'PKatt', 'PKA', 'PKm', 'Save%.1', 'FK', 'CK', 'OG', 'Cmp', 'Att', 
                        'Cmp%', 'Att (GK)', 'Thr', 'Launch%', 'AvgLen', 'Att.1', 'Launch%.1', 
                        'AvgLen.1', 'Opp', 'Stp%']

# Unisco i file Excel
df1 = pd.read_excel(f1)
df2 = pd.read_excel(f2).iloc[1:].reset_index(drop=True)
df3 = pd.read_excel(f3).iloc[1:].reset_index(drop=True)
df4 = pd.read_excel(f4).iloc[1:].reset_index(drop=True)
df_unito = pd.concat([df1, df2, df3, df4], ignore_index=True)

# Rimuovo le colonne inutili
df_unito = df_unito.drop(columns=colonne_da_eliminare, errors='ignore')

df5 = pd.read_excel(f5).iloc[1:].reset_index(drop=True)
df5 = df5.drop(columns=colonne_da_eliminare, errors='ignore')

# Rimuovo le righe con NaN nel dataset unito
df_unito = df_unito.dropna()

# Separare le colonne in input (X) e output (y)
X = df_unito.drop(columns = 'Saves', errors = 'ignore')
y = df_unito['Saves']

# Normalizzazione degli input
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Divisione in set di addestramento e test
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Costruzione della rete neurale
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)  # Modificato il livello di output
])

# Compilazione del modello
model.compile(optimizer='adam', loss='mean_squared_error')

# Addestramento del modello
model.fit(X_train, y_train, epochs=1000, batch_size=32, validation_split=0.2)

# Pre-processamento dei nuovi dati (df5)
df_new = df5.dropna()  # Rimuovo le righe con NaN nei nuovi dati

# Verifica che le colonne siano allineate con il DataFrame di addestramento
X_new = df_new[X.columns]

# Pre-elabora i dati nuovi
X_new_scaled = scaler.transform(X_new)

# Usa il modello per fare previsioni
y_pred = model.predict(X_new_scaled)

# Sostituisci i valori negativi con 0
y_pred = np.clip(y_pred, 0, None)

# Visualizza le previsioni
df_new['Pred_Saves'] = y_pred[:, 0]

print(df_new[['Saves', 'Pred_Saves']])

# Calcola separatamente la somma delle previsioni per le parate
somma_saves_attesi = df_new['Pred_Saves'].sum()

# Calcola separatamente la somma effettiva delle parate nei dati originali
somma_saves_effettivi = df_new['Saves'].sum()

In [ ]:
# Calcola separatamente la somma delle previsioni per Gls e Ast
somma_goal_attesi = df_new['Pred_Saves'].sum()

# Calcola separatamente la somma effettiva dei goal e assist nei dati originali
somma_goal = df_new['Saves'].sum()

# Risultati dei gol
print(f"La somma delle parate è {somma_saves_effettivi}")
print(f"La somma dei goal ottenuti dalla rete neurale è {somma_saves_attesi}")

In [ ]:
# Errore

diff_saves = abs(somma_saves_effettivi - somma_saves_attesi)

# Trovo l'errore medio dividendo per il numero di giocatori 

avg_err_saves = diff_saves/194  

print(f"La differenza fra la somma effettiva e quella della rete neurale è di {diff_saves} parate")
print(f"L'errore medio per le parate è {avg_err_saves} per giocatore")